In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
def read_file_create_instances(name,path):
    """
    read the Json file and get the insights 
    name - name of JSON file
    path - list that represent the path to the instances (here we assume theres only one nested feature)
    """
    
    with open(name,"r", encoding="utf-8") as f:
        data = json.load(f)
        video_instances = data[path[0]][0][path[1]]
        return video_instances
  

In [3]:
PodcastID = 'biqawi_#27'
instances = read_file_create_instances(PodcastID +'.json',path = ['videos','insights'])

In [4]:
def create_custom_data_frame(data,features):
    """
    given data and fetures, making a data frame and visualizing it.
    """
    df = pd.DataFrame(data = data,columns=features)
    rows = df.shape[0]
    return df.head(rows)

In [5]:
def add_column(samples,feature_name,sub_features_names):
    """
    creating data frame by feature
    """
    feature =  pd.json_normalize(samples, record_path = [feature_name])
    feature_df = create_custom_data_frame(data = feature,features=sub_features_names)
    return feature_df

In [6]:
transcript = add_column(instances,'transcript',["speakerId", "text", "confidence"])
num_samples = transcript.shape[0]
add_column(instances,'transcript',["speakerId", "text", "confidence"])

,speakerId,text,confidence
0,1,غضب إنه مش بس غضبنا، هذا الناس لازم يفهموه غضب...,0.7271
1,1,أنا ما تهجرت بس جدي تهجر لما أحكي عن إيه كيف ا...,0.7188
2,1,فالتجارب إللي جبت من أجيال هي إللي تعطينا الغض...,0.7825
3,1,أكبر يعني أنا كبني آدم كشخص واحد أنا ما بهم كم...,0.8014
4,2,مرحبا وأهلا وسهلا فيكم في حلقة جديدة من بودكاس...,0.7807
...,...,...,...
289,2,طيب يا عدن شكرا جزيلا لإلك آه أنا يعني كثير مب...,0.7240
290,2,مع البدوان مع النقب يعني هيك و البدوان بتعرفي ...,0.6801
291,1,يعني من يوم يومهم من ال 48 واحنا جاهزين يلا.,0.6801
292,2,يا هاهاها فلإلي كان جدا مهم. شكرا على المقابلة...,0.6801


In [7]:
#creating samples of each sentences' time frame
instances_temp = []
for i in range(num_samples):
    instances_temp.append(instances['transcript'][i]['instances'][0])

features = ['start', 'end']
instances_temp = pd.json_normalize(data = instances_temp)
instances_df = create_custom_data_frame(data = instances_temp,features=features)

In [8]:
#creating Dataframe, merge the transcript  and time stemps 
dataframe = pd.concat([transcript,instances_df],axis = 1, join = 'inner')
# need to change name to fit the Json file name:
dataframe['PodcastID'] = PodcastID
dataframe.head(num_samples)

,speakerId,text,confidence,start,end,PodcastID
0,1,غضب إنه مش بس غضبنا، هذا الناس لازم يفهموه غضب...,0.7271,0:00:00.09,0:00:31.82,biqawi_#27
1,1,أنا ما تهجرت بس جدي تهجر لما أحكي عن إيه كيف ا...,0.7188,0:00:32.03,0:00:42.41,biqawi_#27
2,1,فالتجارب إللي جبت من أجيال هي إللي تعطينا الغض...,0.7825,0:00:43.03,0:00:57.66,biqawi_#27
3,1,أكبر يعني أنا كبني آدم كشخص واحد أنا ما بهم كم...,0.8014,0:00:58.32,0:01:06.64,biqawi_#27
4,2,مرحبا وأهلا وسهلا فيكم في حلقة جديدة من بودكاس...,0.7807,0:01:20.16,0:01:50.96,biqawi_#27
...,...,...,...,...,...,...
289,2,طيب يا عدن شكرا جزيلا لإلك آه أنا يعني كثير مب...,0.7240,0:52:51.8,0:53:26.18,biqawi_#27
290,2,مع البدوان مع النقب يعني هيك و البدوان بتعرفي ...,0.6801,0:53:26.44,0:53:33.25,biqawi_#27
291,1,يعني من يوم يومهم من ال 48 واحنا جاهزين يلا.,0.6801,0:53:33.26,0:53:35.72,biqawi_#27
292,2,يا هاهاها فلإلي كان جدا مهم. شكرا على المقابلة...,0.6801,0:53:36.2,0:53:47.15,biqawi_#27


In [9]:
#creating column emotions, non tagged sample gets None, econfidence  - confidence of the tagged emotion
emotions = pd.json_normalize(instances, record_path = ['emotions'])
features_name = ["type", "instances"]
emotions_df = create_custom_data_frame(data = emotions,features=features_name)
dataframe['emotion'] = np.full(num_samples,None)
dataframe['econfidence'] = np.full(num_samples,float(0))
index = 0
for j in range(len(emotions_df.type)):
    for i in range(len(emotions_df.instances[j])):
        instance = emotions_df.instances[j][i]
        index = int(np.where(dataframe['start'] ==  instance['start'])[0])
        dataframe['emotion'][index] = emotions_df.type[j]
        dataframe['econfidence'][index] = emotions_df.instances[j][i]['confidence']

In [10]:
sorted_df = dataframe.drop(columns=['emotion', 'econfidence'])
sorted_df

,speakerId,text,confidence,start,end,PodcastID
0,1,غضب إنه مش بس غضبنا، هذا الناس لازم يفهموه غضب...,0.7271,0:00:00.09,0:00:31.82,biqawi_#27
1,1,أنا ما تهجرت بس جدي تهجر لما أحكي عن إيه كيف ا...,0.7188,0:00:32.03,0:00:42.41,biqawi_#27
2,1,فالتجارب إللي جبت من أجيال هي إللي تعطينا الغض...,0.7825,0:00:43.03,0:00:57.66,biqawi_#27
3,1,أكبر يعني أنا كبني آدم كشخص واحد أنا ما بهم كم...,0.8014,0:00:58.32,0:01:06.64,biqawi_#27
4,2,مرحبا وأهلا وسهلا فيكم في حلقة جديدة من بودكاس...,0.7807,0:01:20.16,0:01:50.96,biqawi_#27
...,...,...,...,...,...,...
289,2,طيب يا عدن شكرا جزيلا لإلك آه أنا يعني كثير مب...,0.7240,0:52:51.8,0:53:26.18,biqawi_#27
290,2,مع البدوان مع النقب يعني هيك و البدوان بتعرفي ...,0.6801,0:53:26.44,0:53:33.25,biqawi_#27
291,1,يعني من يوم يومهم من ال 48 واحنا جاهزين يلا.,0.6801,0:53:33.26,0:53:35.72,biqawi_#27
292,2,يا هاهاها فلإلي كان جدا مهم. شكرا على المقابلة...,0.6801,0:53:36.2,0:53:47.15,biqawi_#27


In [12]:
def create_text_file(df, filename):
    # Open a file to write
    with open  (filename, 'w', encoding='utf-8') as file:
        for text in df['text']:
            #adding two lines between each sentence
            file.write(text + '\n\n')
    


create_text_file(sorted_df, 'output_text_file.txt')